In [1]:
# using ast.literal_eval()
import sys
import ast
import os
import time
import pandas as pd
import pickle
import glob
import numpy as np
import statistics
from scipy.stats import mannwhitneyu
import math

In [2]:
with open("../wgcna_tutorial/unweighted_pan_tissue_paths.dictionary", 'rb') as config_dictionary_file:
 
    # Step 3
    paths = pickle.load(config_dictionary_file)

In [3]:
# create list
file2 = "common_nodes_hPPIN_STRING_PANCAN.txt"
combined_list = []
with open(file2) as IN:
    for line in IN:
        combined_list.append(line.rstrip().split("\t")[0])

In [4]:
def compute_sp(list1, list2, paths):
    total_sp = []
    for i in range(len(list1)):
        for j in range(len(list2)):
            if(list1[i] in paths.keys()):
                shortest_paths = paths[list1[i]]
                if(list2[j] in shortest_paths.keys()):
                    total_sp.append(shortest_paths[list2[j]])
    return({"distribution": total_sp,
            "mean": statistics.mean(total_sp),
            "median": statistics.median(total_sp)})

In [5]:
# Read Cancersea Data
folder_path = "/data/timonaj/cancer_as_wound/ppi_analysis/hallmark"

filenames = os.listdir(folder_path)

hallmark = {}

for filename in filenames:
    file_path = f"{folder_path}/{filename}"
    hallmark[filename[:-4]] = list()

    with open(file_path) as IN:
        for line in IN:
            hallmark[filename[:-4]].append(line.rstrip())

In [6]:
hallmark["COSMIC"] = list()
with open("/data/timonaj/cancer_as_wound/ppi_analysis/cosmic_hallmarks.txt") as IN:
    for line in IN:
        hallmark["COSMIC"].append(line.rstrip().split("\t")[0])

In [7]:
# read in HP sets
folder_path = "/data/timonaj/cancer_as_wound/ppi_analysis/HP_genesets/"

filenames = os.listdir(folder_path)

hp_genesets = {}

for filename in filenames:
    file_path = f"{folder_path}/{filename}"
    hp_genesets[filename[:-4]] = list()

    with open(file_path) as IN:
        for line in IN:
            hp_genesets[filename[:-4]].append(line.rstrip().split("\t")[0])

In [8]:
# read in HP matched controls set
folder_path = "/data/timonaj/cancer_as_wound/ppi_analysis/HP_genesets_matched_controls/"

filenames = glob.glob("/data/timonaj/cancer_as_wound/ppi_analysis/HP_genesets_matched_controls/pan_tissue*")
hp_genesets_matched = {}

for filename in filenames:
    name = filename.split("pan_tissue_")[1][:-4]
    hp_genesets_matched[name] = list()

    with open(filename) as IN:
        for line in IN:
            hp_genesets_matched[name].append(line.rstrip().split("\t")[0])

In [9]:
a_mean = np.zeros(shape=(len(hp_genesets), len(hallmark)))
a_median = np.zeros(shape=(len(hp_genesets), len(hallmark)))
a_dist = pd.DataFrame(index = range(len(hallmark)),
                      columns = range(len(hp_genesets)))

In [10]:
i=0
for key1,value1 in hp_genesets.items():
    j=0
    for key2,value2 in hallmark.items():
        current_info = compute_sp(value1, value2, paths)
        a_mean[i][j] = current_info["mean"]
        a_median[i][j] = current_info["median"]
        a_dist[i][j] = current_info["distribution"]
        j+=1
    i+=1

In [11]:
pd.DataFrame(a_median)

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,5.0,4.0,4.0,4.0,5.0,4.0,4.0,5.0,4.0,5.0,...,4.0,4.0,5.0,4.0,5.0,4.0,4.0,4.0,4.0,4.0
1,5.0,5.0,5.0,4.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,4.0,5.0,5.0,5.0,5.0,4.0,5.0,5.0,5.0
2,5.0,4.0,4.0,4.0,5.0,5.0,4.0,5.0,4.0,5.0,...,4.0,4.0,5.0,4.0,5.0,5.0,4.0,4.0,5.0,4.0
3,5.0,4.0,5.0,4.0,5.0,5.0,4.0,5.0,5.0,5.0,...,4.0,4.0,5.0,4.0,5.0,5.0,4.0,4.0,5.0,4.0
4,5.0,5.0,5.0,4.0,5.0,5.0,4.0,5.0,5.0,5.0,...,5.0,4.0,5.0,4.0,5.0,5.0,4.0,4.0,5.0,4.0
5,5.0,5.0,5.0,4.0,5.0,5.0,4.0,5.0,5.0,5.0,...,4.0,4.0,5.0,4.0,5.0,5.0,4.0,4.0,5.0,4.0
6,5.0,5.0,5.0,4.0,5.0,5.0,4.0,5.0,5.0,5.0,...,4.0,4.0,5.0,4.0,5.0,5.0,4.0,4.0,5.0,4.0
7,5.0,5.0,4.0,4.0,5.0,5.0,4.0,5.0,4.0,5.0,...,5.0,4.0,5.0,5.0,5.0,5.0,4.0,5.0,5.0,4.0
8,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,4.0,5.0,5.0,5.0,5.0,4.0,5.0,5.0,5.0


In [12]:
pd.DataFrame(a_mean)

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,4.810123,4.464491,4.466109,4.089706,4.701504,4.488057,4.090918,4.837699,4.468424,4.632960,...,4.427641,4.396859,4.607552,4.125214,4.871600,4.461501,4.294061,4.433590,4.546451,4.413507
1,4.949020,4.756438,4.600030,4.506357,4.829090,4.700772,4.599827,4.931062,4.605183,4.798644,...,4.624318,4.549233,4.704360,4.586328,4.948902,4.692402,4.513937,4.792701,4.765105,4.592010
2,4.885852,4.535080,4.547161,4.300036,4.789373,4.608005,4.387849,4.941244,4.534318,4.725425,...,4.503678,4.487373,4.690846,4.372200,4.962395,4.603589,4.396203,4.475689,4.713156,4.491862
3,4.876963,4.473186,4.631626,4.205332,4.799333,4.583779,4.259007,4.985688,4.609269,4.673943,...,4.466189,4.444642,4.711580,4.276610,5.001876,4.569776,4.345564,4.396011,4.670295,4.472035
4,4.999419,4.655223,4.710255,4.458557,4.899432,4.734413,4.564901,5.029044,4.690333,4.828776,...,4.616927,4.580603,4.799859,4.543068,5.057151,4.732227,4.514125,4.587121,4.830684,4.593685
5,4.897702,4.618995,4.556381,4.326095,4.801639,4.643271,4.380907,4.936988,4.555857,4.745735,...,4.538442,4.545447,4.715632,4.388351,4.997109,4.623116,4.406773,4.634704,4.738034,4.555065
6,4.909668,4.678669,4.654988,4.408331,4.807830,4.667864,4.486874,4.940300,4.647469,4.772190,...,4.571022,4.503831,4.722324,4.481476,4.965643,4.656005,4.459839,4.676026,4.726993,4.571218
7,4.874940,4.744621,4.497649,4.438550,4.695260,4.618960,4.524511,4.723023,4.512347,4.776352,...,4.578268,4.450280,4.592555,4.513015,4.746677,4.614764,4.487090,4.759442,4.638584,4.515740
8,4.920458,4.829927,4.705844,4.588333,4.808714,4.733759,4.689022,4.873771,4.717828,4.805321,...,4.652413,4.531926,4.730110,4.673562,4.919747,4.719302,4.559719,4.890225,4.762941,4.651538


In [13]:
pd.DataFrame(a_mean, columns = hallmark.keys(), index = hp_genesets.keys()).to_csv("pan_tissue_meanSP_HP_Phenotypes_hallmark.csv")

In [14]:
a_matched_mean = np.zeros(shape=(len(hp_genesets_matched), len(hallmark)))
a_matched_median = np.zeros(shape=(len(hp_genesets_matched), len(hallmark)))
a_matched_dist = pd.DataFrame(index = range(len(hallmark)),
                              columns = range(len(hp_genesets_matched)))

In [15]:
i=0
for key1,value1 in hp_genesets_matched.items():
    j=0
    for key2,value2 in hallmark.items():
        current_info = compute_sp(value1, value2, paths)
        a_matched_mean[i][j] = current_info["mean"]
        a_matched_median[i][j] = current_info["median"]
        a_matched_dist[i][j] = current_info["distribution"]
        j+=1
    i+=1

In [16]:
pd.DataFrame(a_matched_median)

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
1,5.0,5.0,4.0,5.0,5.0,5.0,5.0,5.0,4.0,5.0,...,5.0,4.0,4.0,5.0,5.0,5.0,4.0,5.0,5.0,4.0
2,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
3,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,4.0,4.0,5.0,5.0,5.0,4.0,5.0,5.0,4.0
4,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
5,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
6,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,4.0,5.0,5.0,5.0,5.0,4.0,5.0,5.0,5.0
7,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,4.0,5.0,5.0,5.0,5.0,4.0,5.0,5.0,5.0
8,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0


In [17]:
pd.DataFrame(a_matched_mean)

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,4.965586,4.914068,4.652507,4.610960,4.794968,4.740801,4.727519,4.843392,4.671438,4.885929,...,4.710135,4.562982,4.675212,4.695020,4.857667,4.725503,4.605921,4.972840,4.733014,4.629008
1,4.927327,4.867616,4.563428,4.569475,4.747763,4.690294,4.679033,4.753157,4.588686,4.838659,...,4.668064,4.512228,4.608331,4.659312,4.761611,4.672926,4.568880,4.931942,4.684359,4.578038
2,5.046210,4.975904,4.655066,4.657287,4.866492,4.815290,4.752552,4.878503,4.679671,4.962768,...,4.786315,4.661722,4.732246,4.741305,4.927231,4.785886,4.687461,5.040087,4.796708,4.696622
3,4.883804,4.888124,4.548584,4.608624,4.698792,4.681511,4.740222,4.687175,4.578257,4.827135,...,4.656470,4.497793,4.575220,4.709335,4.719280,4.669847,4.563337,4.956867,4.675078,4.575534
4,5.044877,5.015725,4.704966,4.722686,4.871216,4.830947,4.834813,4.880142,4.730265,4.973876,...,4.790791,4.663045,4.754578,4.804555,4.909370,4.805335,4.697422,5.082618,4.838007,4.718158
5,4.945907,4.898865,4.583365,4.605100,4.771689,4.720522,4.719452,4.768152,4.610022,4.856148,...,4.692466,4.533684,4.640156,4.695639,4.793519,4.706094,4.594052,4.954903,4.714286,4.610755
6,4.939206,4.885022,4.588081,4.597173,4.787144,4.727671,4.698509,4.787585,4.615682,4.847046,...,4.676145,4.567984,4.652453,4.675982,4.844580,4.696228,4.570406,4.980861,4.721102,4.614939
7,4.956938,4.885514,4.639541,4.580151,4.804224,4.733238,4.675811,4.835950,4.664390,4.864674,...,4.695634,4.563686,4.689059,4.663093,4.870691,4.710952,4.590641,4.945007,4.727470,4.631894
8,5.099958,5.040545,4.724098,4.752367,4.913858,4.844298,4.870937,4.907919,4.744690,5.035908,...,4.859484,4.640598,4.752741,4.846270,4.896345,4.851399,4.768737,5.051939,4.827012,4.737268


In [18]:
pd.DataFrame(a_matched_mean, columns = hallmark.keys(), index = hp_genesets_matched.keys()).to_csv("pan_tissue_meanSP_HPmatched_Phenotypes_hallmark.csv")

In [19]:
mannwhitneyu_p = np.zeros(shape=(len(hp_genesets), len(hallmark)))
test_dist = np.zeros(shape=(len(hp_genesets), len(hallmark)))
test_matched_dist = np.zeros(shape=(len(hp_genesets), len(hallmark)))

In [20]:
LogFC = np.zeros(shape=(len(hp_genesets), len(hallmark)))
for i in range(len(hp_genesets)):
    for j in range(len(hallmark)):
        actual_dist = a_dist[i][j]
        control_dist = a_matched_dist[i][j]
        res = mannwhitneyu(actual_dist, control_dist, alternative="less")
        mannwhitneyu_p[i][j] = res.pvalue
        test_dist[i][j] = statistics.mean(actual_dist)
        test_matched_dist[i][j] = statistics.mean(control_dist)
        LogFC[i][j] = math.log((test_dist[i][j]/test_matched_dist[i][j]))

In [21]:
hallmark.keys()

dict_keys(['PROTEIN_SECRETION', 'COAGULATION', 'E2F_TARGETS', 'COMPLEMENT', 'UNFOLDED_PROTEIN_RESPONSE', 'UV_RESPONSE_UP', 'IL6_JAK_STAT3_SIGNALING', 'MYC_TARGETS_V2', 'G2M_CHECKPOINT', 'XENOBIOTIC_METABOLISM', 'ESTROGEN_RESPONSE_EARLY', 'HEME_METABOLISM', 'WNT_BETA_CATENIN_SIGNALING', 'APOPTOSIS', 'KRAS_SIGNALING_UP', 'APICAL_SURFACE', 'NOTCH_SIGNALING', 'TNFA_SIGNALING_VIA_NFKB', 'GLYCOLYSIS', 'APICAL_JUNCTION', 'IL2_STAT5_SIGNALING', 'P53_PATHWAY', 'PANCREAS_BETA_CELLS', 'BILE_ACID_METABOLISM', 'INTERFERON_ALPHA_RESPONSE', 'HEDGEHOG_SIGNALING', 'ALLOGRAFT_REJECTION', 'MITOTIC_SPINDLE', 'TGF_BETA_SIGNALING', 'UV_RESPONSE_DN', 'KRAS_SIGNALING_DN', 'PEROXISOME', 'FATTY_ACID_METABOLISM', 'EPITHELIAL_MESENCHYMAL_TRANSITION', 'SPERMATOGENESIS', 'MTORC1_SIGNALING', 'CHOLESTEROL_HOMEOSTASIS', 'OXIDATIVE_PHOSPHORYLATION', 'MYOGENESIS', 'INTERFERON_GAMMA_RESPONSE', 'HYPOXIA', 'ANDROGEN_RESPONSE', 'ADIPOGENESIS', 'DNA_REPAIR', 'INFLAMMATORY_RESPONSE', 'MYC_TARGETS_V1', 'PI3K_AKT_MTOR_SIGNALING

In [22]:
pvals = pd.DataFrame(mannwhitneyu_p, columns = hallmark.keys(), index = hp_genesets.keys())

In [23]:
pvals

,PROTEIN_SECRETION,COAGULATION,E2F_TARGETS,COMPLEMENT,UNFOLDED_PROTEIN_RESPONSE,UV_RESPONSE_UP,IL6_JAK_STAT3_SIGNALING,MYC_TARGETS_V2,G2M_CHECKPOINT,XENOBIOTIC_METABOLISM,...,ANDROGEN_RESPONSE,ADIPOGENESIS,DNA_REPAIR,INFLAMMATORY_RESPONSE,MYC_TARGETS_V1,PI3K_AKT_MTOR_SIGNALING,ESTROGEN_RESPONSE_LATE,ANGIOGENESIS,REACTIVE_OXYGEN_SPECIES_PATHWAY,COSMIC
downregulated.wound,7.716702e-39,0.000000e+00,6.855909e-158,0.000000e+00,1.808592e-14,1.155390e-186,0.000000e+00,9.650281e-01,2.716809e-162,4.450341e-179,...,2.767100e-125,3.182245e-90,8.983817e-08,0.000000e+00,1.000000e+00,1.737990e-117,0.000000e+00,1.485946e-117,2.435466e-29,0.000000e+00
upregulated.stress,9.854831e-01,9.027310e-26,1.000000e+00,4.721438e-19,1.000000e+00,9.782818e-01,1.768743e-15,1.000000e+00,1.000000e+00,3.378024e-06,...,2.989707e-05,1.000000e+00,1.000000e+00,1.654596e-26,1.000000e+00,9.969358e-01,4.964958e-15,4.451511e-09,1.000000e+00,9.999999e-01
downregulated.regen,2.534866e-32,1.920390e-233,3.814921e-24,0.000000e+00,1.653283e-12,1.074651e-101,2.138094e-174,9.999998e-01,1.063039e-41,1.268878e-107,...,6.236826e-88,1.661273e-77,3.618462e-04,0.000000e+00,9.999998e-01,1.553167e-42,1.171321e-215,2.729158e-77,1.174743e-07,0.000000e+00
msigdb.wound,7.269255e-02,4.161818e-267,1.000000e+00,0.000000e+00,1.000000e+00,2.618080e-28,0.000000e+00,1.000000e+00,9.999991e-01,1.757972e-65,...,1.124736e-53,4.493201e-17,1.000000e+00,0.000000e+00,1.000000e+00,7.820537e-15,2.567504e-158,2.703368e-92,1.953116e-01,2.900281e-107
msigdb.regen,5.322548e-06,9.667976e-117,6.061331e-01,7.447114e-154,1.930046e-01,2.210670e-23,6.203503e-73,1.000000e+00,1.441060e-04,2.351087e-38,...,8.924219e-31,6.618636e-22,9.487386e-01,3.032295e-137,1.000000e+00,1.394768e-08,8.876446e-73,9.194450e-42,4.560864e-02,1.503221e-105
upregulated.regen,1.455720e-07,1.226972e-132,4.140150e-04,2.422248e-293,9.620295e-01,4.482920e-23,7.750655e-202,1.000000e+00,5.673325e-10,1.640206e-37,...,6.926841e-41,6.129654e-01,1.000000e+00,0.000000e+00,1.000000e+00,5.201714e-13,4.997942e-124,3.815331e-33,8.868629e-01,1.168410e-39
upregulated.wound,1.038474e-04,5.473469e-90,1.000000e+00,1.646763e-152,7.730880e-01,7.499969e-18,2.995869e-88,1.000000e+00,9.979252e-01,7.277727e-26,...,2.653260e-23,5.887679e-18,1.000000e+00,2.293973e-152,1.000000e+00,7.532777e-07,3.028957e-55,8.605129e-38,4.610813e-01,4.512609e-34
msigdb.stress,1.214032e-08,3.229616e-44,3.284283e-72,8.078379e-77,1.710004e-20,9.282387e-35,7.465141e-41,1.314943e-16,1.110839e-78,2.637797e-20,...,2.927617e-19,4.273902e-37,2.874523e-21,9.488799e-83,1.417793e-58,9.075450e-15,3.712986e-30,8.719908e-17,6.763753e-06,3.685310e-140
downregulated.stress,3.281697e-21,1.606787e-30,4.131261e-03,1.501889e-43,5.402577e-13,5.753394e-18,1.004119e-25,9.136707e-03,2.428614e-03,3.855311e-53,...,3.777821e-25,1.552213e-20,1.073965e-02,3.418843e-41,3.510892e-01,2.105095e-13,8.130567e-55,3.188040e-04,2.201160e-03,1.161896e-36


In [24]:
pvals.to_csv("pan_tissue_matched_pvals_hallmark.csv")

In [25]:
LFC = pd.DataFrame(LogFC, columns = hallmark.keys(), index = hp_genesets.keys())

In [26]:
LFC

,PROTEIN_SECRETION,COAGULATION,E2F_TARGETS,COMPLEMENT,UNFOLDED_PROTEIN_RESPONSE,UV_RESPONSE_UP,IL6_JAK_STAT3_SIGNALING,MYC_TARGETS_V2,G2M_CHECKPOINT,XENOBIOTIC_METABOLISM,...,ANDROGEN_RESPONSE,ADIPOGENESIS,DNA_REPAIR,INFLAMMATORY_RESPONSE,MYC_TARGETS_V1,PI3K_AKT_MTOR_SIGNALING,ESTROGEN_RESPONSE_LATE,ANGIOGENESIS,REACTIVE_OXYGEN_SPECIES_PATHWAY,COSMIC
downregulated.wound,-0.031809,-0.095947,-0.040889,-0.119963,-0.019685,-0.054786,-0.144631,-0.001176,-0.044431,-0.053164,...,-0.061850,-0.037086,-0.014578,-0.129385,0.002864,-0.057489,-0.070110,-0.114781,-0.040215,-0.047673
upregulated.stress,0.004393,-0.023105,0.007989,-0.013909,0.016985,0.002232,-0.017073,0.036745,0.003589,-0.008304,...,-0.009416,0.008168,0.020624,-0.015788,0.038580,0.004159,-0.012098,-0.028639,0.017090,0.003047
downregulated.regen,-0.032294,-0.092764,-0.023453,-0.079810,-0.015974,-0.044001,-0.079843,0.012779,-0.031553,-0.049006,...,-0.060866,-0.038118,-0.008787,-0.081046,0.007111,-0.038835,-0.064150,-0.118763,-0.017572,-0.044576
msigdb.wound,-0.001402,-0.088708,0.018092,-0.091576,0.021171,-0.021097,-0.107048,0.061741,0.006751,-0.032250,...,-0.041722,-0.011887,0.029369,-0.096386,0.058157,-0.021662,-0.048899,-0.120076,-0.001024,-0.022880
msigdb.regen,-0.009052,-0.074588,0.001123,-0.057552,0.005776,-0.020185,-0.057446,0.030056,-0.008478,-0.029606,...,-0.036966,-0.017838,0.009479,-0.055962,0.029658,-0.015331,-0.039802,-0.102574,-0.001515,-0.026736
upregulated.regen,-0.009794,-0.058827,-0.005905,-0.062499,0.006257,-0.016500,-0.074437,0.034797,-0.011819,-0.022999,...,-0.033374,0.002591,0.016135,-0.067681,0.041595,-0.017789,-0.041620,-0.066805,0.005025,-0.012152
upregulated.wound,-0.005998,-0.043160,0.014477,-0.041945,0.004312,-0.012731,-0.046089,0.031400,0.006863,-0.015564,...,-0.022737,-0.014144,0.014906,-0.042487,0.024682,-0.008602,-0.024489,-0.063154,0.001247,-0.009519
msigdb.stress,-0.016680,-0.029263,-0.031061,-0.031404,-0.022942,-0.024440,-0.032893,-0.023629,-0.033140,-0.018323,...,-0.025312,-0.025164,-0.020796,-0.032713,-0.025791,-0.020629,-0.022815,-0.038248,-0.018981,-0.025397
downregulated.stress,-0.035831,-0.042683,-0.003872,-0.035126,-0.021630,-0.023083,-0.038062,-0.006982,-0.005678,-0.046870,...,-0.043546,-0.023696,-0.004773,-0.036288,0.004768,-0.027606,-0.044820,-0.032534,-0.013362,-0.018263


In [27]:
LFC.to_csv("pan_tissue_matched_LFC_hallmark.csv")

In [28]:
pd.DataFrame(test_dist)

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,4.810123,4.464491,4.466109,4.089706,4.701504,4.488057,4.090918,4.837699,4.468424,4.632960,...,4.427641,4.396859,4.607552,4.125214,4.871600,4.461501,4.294061,4.433590,4.546451,4.413507
1,4.949020,4.756438,4.600030,4.506357,4.829090,4.700772,4.599827,4.931062,4.605183,4.798644,...,4.624318,4.549233,4.704360,4.586328,4.948902,4.692402,4.513937,4.792701,4.765105,4.592010
2,4.885852,4.535080,4.547161,4.300036,4.789373,4.608005,4.387849,4.941244,4.534318,4.725425,...,4.503678,4.487373,4.690846,4.372200,4.962395,4.603589,4.396203,4.475689,4.713156,4.491862
3,4.876963,4.473186,4.631626,4.205332,4.799333,4.583779,4.259007,4.985688,4.609269,4.673943,...,4.466189,4.444642,4.711580,4.276610,5.001876,4.569776,4.345564,4.396011,4.670295,4.472035
4,4.999419,4.655223,4.710255,4.458557,4.899432,4.734413,4.564901,5.029044,4.690333,4.828776,...,4.616927,4.580603,4.799859,4.543068,5.057151,4.732227,4.514125,4.587121,4.830684,4.593685
5,4.897702,4.618995,4.556381,4.326095,4.801639,4.643271,4.380907,4.936988,4.555857,4.745735,...,4.538442,4.545447,4.715632,4.388351,4.997109,4.623116,4.406773,4.634704,4.738034,4.555065
6,4.909668,4.678669,4.654988,4.408331,4.807830,4.667864,4.486874,4.940300,4.647469,4.772190,...,4.571022,4.503831,4.722324,4.481476,4.965643,4.656005,4.459839,4.676026,4.726993,4.571218
7,4.874940,4.744621,4.497649,4.438550,4.695260,4.618960,4.524511,4.723023,4.512347,4.776352,...,4.578268,4.450280,4.592555,4.513015,4.746677,4.614764,4.487090,4.759442,4.638584,4.515740
8,4.920458,4.829927,4.705844,4.588333,4.808714,4.733759,4.689022,4.873771,4.717828,4.805321,...,4.652413,4.531926,4.730110,4.673562,4.919747,4.719302,4.559719,4.890225,4.762941,4.651538


In [29]:
pd.DataFrame(test_matched_dist)

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,4.965586,4.914068,4.652507,4.610960,4.794968,4.740801,4.727519,4.843392,4.671438,4.885929,...,4.710135,4.562982,4.675212,4.695020,4.857667,4.725503,4.605921,4.972840,4.733014,4.629008
1,4.927327,4.867616,4.563428,4.569475,4.747763,4.690294,4.679033,4.753157,4.588686,4.838659,...,4.668064,4.512228,4.608331,4.659312,4.761611,4.672926,4.568880,4.931942,4.684359,4.578038
2,5.046210,4.975904,4.655066,4.657287,4.866492,4.815290,4.752552,4.878503,4.679671,4.962768,...,4.786315,4.661722,4.732246,4.741305,4.927231,4.785886,4.687461,5.040087,4.796708,4.696622
3,4.883804,4.888124,4.548584,4.608624,4.698792,4.681511,4.740222,4.687175,4.578257,4.827135,...,4.656470,4.497793,4.575220,4.709335,4.719280,4.669847,4.563337,4.956867,4.675078,4.575534
4,5.044877,5.015725,4.704966,4.722686,4.871216,4.830947,4.834813,4.880142,4.730265,4.973876,...,4.790791,4.663045,4.754578,4.804555,4.909370,4.805335,4.697422,5.082618,4.838007,4.718158
5,4.945907,4.898865,4.583365,4.605100,4.771689,4.720522,4.719452,4.768152,4.610022,4.856148,...,4.692466,4.533684,4.640156,4.695639,4.793519,4.706094,4.594052,4.954903,4.714286,4.610755
6,4.939206,4.885022,4.588081,4.597173,4.787144,4.727671,4.698509,4.787585,4.615682,4.847046,...,4.676145,4.567984,4.652453,4.675982,4.844580,4.696228,4.570406,4.980861,4.721102,4.614939
7,4.956938,4.885514,4.639541,4.580151,4.804224,4.733238,4.675811,4.835950,4.664390,4.864674,...,4.695634,4.563686,4.689059,4.663093,4.870691,4.710952,4.590641,4.945007,4.727470,4.631894
8,5.099958,5.040545,4.724098,4.752367,4.913858,4.844298,4.870937,4.907919,4.744690,5.035908,...,4.859484,4.640598,4.752741,4.846270,4.896345,4.851399,4.768737,5.051939,4.827012,4.737268
